# figures galore

results visualizations for paper or something idk

## setup, data loading needed for most visualizations

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import json
import math
import os
from pathlib import Path
from typing import List
import sys

import cartopy
import cartopy.crs as ccrs
import cv2
import dataclass_array as dca
import jax.numpy as jnp
import matplotlib as mpl
import matplotlib.pyplot as plt
import mitsuba as mi
import numpy as np
import open3d as o3d
from PIL import Image
import pandas as pd
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import pycolmap
import pyransac3d as pyrsc
import pyrender
import quaternion
import scipy.interpolate
import seaborn as sns
import trimesh
import visu3d as v3d
import yaml

sys.path.append(os.path.abspath(os.path.join("..", "bop_toolkit")))
from bop_toolkit.bop_toolkit_lib.pose_error import vsd, mssd, mspd
from bop_toolkit.bop_toolkit_lib.misc import get_symmetry_transformations
from bop_toolkit.bop_toolkit_lib.renderer import create_renderer

import burybarrel.colmap_util as cutil
from burybarrel.image import render_v3d, render_models, to_contour, overlay_img_alpha, imgs_from_paths, imgs_from_dir
from burybarrel.camera import RadialCamera, load_v3dcams
from burybarrel.mesh import subdivide_mesh, recolor_mesh, segment_pc_from_mask, segment_pc_from_masks
from burybarrel.transform import T_from_blender, T_from_translation, scale_T_translation, scale_cams, scale_pc, icp, qangle, qmean, closest_quat_sym, get_axes_rot, apply_T_xyz, icp_scaleadapt
from burybarrel.plotting import get_line3d_trace, get_axes_traces, get_plane_zup, get_surface_line_traces, get_ray_trace, generate_domain, get_carree_axis, get_carree_gl, get_trimesh_traces, get_cam_imgsurf_trace, get_resized_pc_trace
from burybarrel.utils import rgb2hex, cmapvals, match_lists, invert_idxs, combine_path_tail, haversine
from burybarrel.estimators import ransac
from burybarrel.foundpose_fit import fitcams, camloss, camcost, qcost, qloss

In [ ]:
# tmpdir = Path("/scratch/jeyan/barreldata/tmp")
tmpdir = Path("/Users/jerry/Projects/ms-stuff/barrel-playground/tmp")
# datarootdir = Path("/scratch/jeyan/barreldata/divedata")
datarootdir = Path("/Users/jerry/Projects/ms-stuff/barrel-playground/barrels/data/input_data")
# resrootdir = Path("/scratch/jeyan/barreldata/results")
resrootdir = Path("/Users/jerry/Projects/ms-stuff/barrel-playground/barrels/results")
# metricscsvpath = Path("/scratch/jeyan/barreldata/results/allmetrics.csv")
metricscsvpath = Path("/Users/jerry/Projects/ms-stuff/barrel-playground/barrels/results/allmetrics.csv")
# metricscsvpath = Path("/Users/jerry/Projects/ms-stuff/barrel-playground/barrels/results/allmetrics-vsdtest.csv")
metricsdf = pd.read_csv(metricscsvpath)
metricsdf.head()

In [ ]:
metricsdf[["avg_vsd", "avg_mssd", "avg_mspd", "burial_error_depth", "avg_dice", "multiview_fitted", "reconstr_type", "pose_type", "use_icp"]].groupby(["multiview_fitted", "use_icp", "pose_type", "reconstr_type"]).mean()

In [ ]:
def filtermetrics(df, name=None, multiview_fitted=None, use_icp=None, pose_type=None, reconstr_type=None):
    if multiview_fitted is not None:
        df = df[df["multiview_fitted"] == multiview_fitted]
    if use_icp is not None:
        df = df[df["use_icp"] == use_icp]
    if pose_type is not None:
        df = df[df["pose_type"] == pose_type]
    if name is not None:
        df = df[df["dataset"] == name]
    if reconstr_type is not None:
        df = df[df["reconstr_type"] == reconstr_type]
    return df

In [ ]:
nsamples = len(filtermetrics(metricsdf, multiview_fitted=True, use_icp=True, pose_type="coarse", reconstr_type="colmap"))
print(f"Number of labeled samples: {nsamples}")

In [ ]:
# loading EVERYTHING
datasetname = "barrelddt1"
singledatarows = metricsdf[metricsdf["dataset"] == datasetname]

# multiview fit, coarse, icp
resrow = filtermetrics(metricsdf, name=datasetname, multiview_fitted=True, use_icp=True, pose_type="coarse", reconstr_type="colmap").iloc[0]
datadir = Path(resrow["datadir"])
datadir = datarootdir / datadir.name
resdir = Path(resrow["resdir"])
resdir = combine_path_tail(resrootdir, resdir, 3)
object_name = resrow["object_name"]

with open("../configs/blender_gt_info.yaml", "rt") as f:
    gtinfo = yaml.safe_load(f)
with open(datadir / "gt_obj2cam.json", "rt") as f:
    gtposes = yaml.safe_load(f)
with open(datadir / "info.json", "rt") as f:
    datainfo = yaml.safe_load(f)
with open(datadir / "camera.json", "rt") as f:
    caminfo = yaml.safe_load(f)
with open(resdir / "../../sam-masks/masksinfo.json", "rt") as f:
    masksinfo = yaml.safe_load(f)
posepath = resdir / "estimated-poses.json"
estinfopath = resdir / "reconstruction-info.json"
with open(posepath, "rt") as f:
    ests = yaml.safe_load(f)
with open(estinfopath, "rt") as f:
    estinfo = yaml.safe_load(f)
with open(resdir / "../../foundpose-output/inference/estimated-poses.json", "rt") as f:
    foundposeests = yaml.safe_load(f)
cams, imgpaths = load_v3dcams(resdir / "../../colmap-out/cam_poses.json", img_parent=datadir / "rgb")
imgs = imgs_from_paths(imgpaths)
names = [imgpath.stem for imgpath in imgpaths]

densepath = resdir / "../../openmvs-out/scene_dense_trimeshvalid.ply"
densetrimesh: trimesh.PointCloud = trimesh.load_mesh(densepath)
densepc = v3d.Point3d(p=densetrimesh.vertices, rgb=densetrimesh.colors[:, :3])

camspec = RadialCamera.from_jsonargs(**caminfo)
# objdir = Path("/scratch/jeyan/barreldata/models3d")
objdir = Path("/Users/jerry/Projects/ms-stuff/barrel-playground/models3d")
objpath = objdir / object_name
with open(objdir / "model_info.json", "rt") as f:
    objinfo = yaml.safe_load(f)
mesh: trimesh.Trimesh = trimesh.load_mesh(objpath)
meshpc = v3d.Point3d(p=mesh.vertices, rgb=[255, 0, 0])

In [ ]:
def get_bestmask(name, masksinfo):
    for maskinfo in masksinfo:
        if Path(maskinfo["img_path"]).stem == name:
            singlemaskinfo = maskinfo
            break
    bestidx = singlemaskinfo["best_idx"]
    # [x_min, y_min, x_max, y_max]
    bbox = list(map(int, singlemaskinfo["bboxes"][bestidx]))
    mask = np.array(Image.open(combine_path_tail(resrootdir, singlemaskinfo["mask_paths"][bestidx], 4)).convert("L"))
    return mask, bbox

## Overall metric stuff

In [ ]:
barrelddt1_gtdepths = [0.015032915702566596, 0.09818726159695168, 0.1374041827193896, 0.026458748898193996, 0.09052084932689697, 0.12932415169126]
barrelddt1_preddepths = [0.10366831265961662, 0.12824764366323105, 0.07549221603803624, 0.18177893748478946, 0.11903873568486345, 0.1500978468339944]
print(f"gt: {np.mean(barrelddt1_gtdepths)}, {np.std(barrelddt1_gtdepths, ddof=1)}")
print(f"pred: {np.mean(barrelddt1_preddepths)}, {np.std(barrelddt1_preddepths, ddof=1)}")

In [ ]:
barrel2_gtdepths = [0.29335228303751276, 0.34200922363306346, 0.24563754728755416, 0.21293290544447796, 0.27697552313441487, 0.36739898508601676]
barrel2_preddepths = [0.30709878688657344, 0.2561838479341959, 0.27318166286843965, 0.22896163462737362, 0.18911786570616762, 0.2382856298925286]
print(f"gt: {np.mean(barrel2_gtdepths)}, {np.std(barrel2_gtdepths, ddof=1)}")
print(f"pred: {np.mean(barrel2_preddepths)}, {np.std(barrel2_preddepths, ddof=1)}")

In [ ]:
barrel4_gtdepths = [0.4704799345008879, 0.40819391359403, 0.5503122869367747, 0.38833767371892347, 0.4562508267884837, 0.378052910596107]
barrel4_preddepths = [0.33413568136126726, 0.3167304009325044, 0.43955146610328766, 0.30348921040778687, 0.3817764489933768, 0.1680248938633282]
print(f"gt: {np.mean(barrel4_gtdepths)}, {np.std(barrel4_gtdepths, ddof=1)}")
print(f"pred: {np.mean(barrel4_preddepths)}, {np.std(barrel4_preddepths, ddof=1)}")

In [ ]:
depthcharge_gtdepths = [0.26347397028114744, 0.3485749353151665, 0.31823952925194576, 0.35069204081037114, 0.32724359991889324, 0.30176044767556437]
depthcharge_preddepths = [0.30201229898659504, 0.3171079900132318, 0.32574205905060366, 0.3941254138345936, 0.1811391822897317, 0.08433619317977994]
print(f"gt: {np.mean(depthcharge_gtdepths)}, {np.std(depthcharge_gtdepths, ddof=1)}")
print(f"pred: {np.mean(depthcharge_preddepths)}, {np.std(depthcharge_preddepths, ddof=1)}")

In [ ]:
smokefloat_gtdepths = [0.34796616686331144, 0.351192928794356, 0.38017098075573125, 0.3552042984040059, 0.3309834707910568, 0.3609907484106306]
smokefloat_preddepths = [0.36471342007886404, 0.3092563092792329, 0.29896591363509645, 0.2220379886707352, 0.3544188161413274, 0.30641872148881755]
print(f"gt: {np.mean(smokefloat_gtdepths)}, {np.std(smokefloat_gtdepths, ddof=1)}")
print(f"pred: {np.mean(smokefloat_preddepths)}, {np.std(smokefloat_preddepths, ddof=1)}")

## gifs?

In [ ]:
fitimgpaths, fitimgs = imgs_from_dir(resdir / "fit-overlays", sortnames=True)
fitimgrgb = []
fitimgprim = []
for i, fitimgpath in enumerate(fitimgpaths):
    if "primitives" in fitimgpath.name:
        fitimgprim.append(fitimgs[i])
    else:
        fitimgrgb.append(fitimgs[i])
allcombined = []
for rgb, prim in zip(fitimgrgb, fitimgprim):
    combined = np.array(rgb)
    combined = np.concatenate((combined, prim), axis=1)
    combined = Image.fromarray(combined)
    allcombined.append(combined)
allcombined[0].save(
    tmpdir / f"fit-overlays-{datasetname}.gif",
    save_all=True,
    append_images=allcombined[1:],
    duration=500,
    loop=0
)

## some 3d overlay wireframe visualization thing

In [ ]:
estidx = 5
gtidx = estidx
camidx = estidx
# note these might be inconsistent indices, but it would depend on SAM failing
est = ests[estidx]
gt = gtposes[gtidx]
colmapcam = cams[camidx]
singlemasksinfo = next(filter(lambda x: Path(x["img_path"]).stem == Path(est["img_path"]).stem, masksinfo))
estscale = estinfo["scalefactor"]
reconstr_mesh: trimesh.Trimesh = trimesh.load(resdir / "../../openmvs-out/scene_dense_mesh_refine.ply")
# reconstr_mesh.vertices = reconstr_mesh.vertices * estscale
bestidx = singlemasksinfo["best_idx"]
mask = np.array(Image.open(combine_path_tail(resrootdir, singlemasksinfo["mask_paths"][bestidx], 4)).convert("L"))
# est, gt, cam, mask
est["img_path"], gt["img_path"], imgpaths[camidx], singlemasksinfo["img_path"]

In [ ]:
# meshvertmask = np.zeros(reconstr_mesh.vertices.shape[0], dtype=bool)
# flooridxsmesh = invert_idxs(segment_pc_from_mask(reconstr_mesh.vertices, mask, colmapcam), reconstr_mesh.vertices.shape[0])
# meshvertmask[flooridxsmesh] = True
# reconstr_floor_mesh = reconstr_mesh.copy()
# # reconstr_floor_mesh.update_vertices(meshvertmask)
# face_mask = meshvertmask[reconstr_mesh.faces].all(axis=1)
# reconstr_floor_mesh.update_faces(face_mask)

In [ ]:
meshcol = np.array([255, 0, 0, 255], dtype=np.uint8)
planecol = np.array([0, 0, 255, 255], dtype=np.uint8)
black = np.array([0, 0, 0, 255], dtype=np.uint8)
transparent = np.array([0, 0, 0, 0], dtype=np.uint8)
mesh = recolor_mesh(mesh, meshcol)
plane = trimesh.creation.box(extents=(100, 100, 0.01))
plane = subdivide_mesh(plane, 7)
plane = recolor_mesh(plane, planecol)

In [ ]:
obj2cam = v3d.Transform(R=est["R"], t=est["t"])
floor2cam = v3d.Transform(R=est["R_floor"], t=est["t_floor"])
obj2camgt = v3d.Transform(R=gt["R"], t=gt["t"])
floor2camgt = v3d.Transform(R=gt["R_floor"], t=gt["t_floor"])

In [ ]:
cam = v3d.Camera(spec=camspec, world_from_cam=v3d.Transform.identity())

In [ ]:
# [xmin, ymin, width, height]
crop = [0, 120, 1920, 875]
pilcrop = [crop[0], crop[1], crop[0] + crop[2], crop[1] + crop[3]]
bestidx = singlemasksinfo["best_idx"]
maskalpha = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGBA)
maskalpha[~(maskalpha[..., 0] > 0), 3] = 0

# predicted code
maskrgb, _, _ = render_models(cam, [mesh, plane], [obj2cam, floor2cam], flags=pyrender.RenderFlags.FLAT)
objmask = np.all(maskrgb == [255, 0, 0], axis=-1)
rgb_wireframe, _, _ = render_models(cam, [recolor_mesh(plane, black)], [floor2cam], wireframe=True, flags=pyrender.RenderFlags.RGBA | pyrender.RenderFlags.FLAT)
rgb_wireframe = rgb_wireframe * ~objmask[..., None]
rgb_wireframe[rgb_wireframe[..., 3] > 0, 3] = 255
rgb_primoverlay, _, _ = render_models(cam, [mesh, plane], [obj2cam, floor2cam], light_intensity=50, wireframe=False)
# Image.fromarray(rgb_wireframe).save(tmpdir / "rendered_wireframe.png")
# Image.fromarray(rgb_primoverlay).save(tmpdir / "rendered.png")
# img = cv2.cvtColor(cv2.imread(est["img_path"]), cv2.COLOR_BGR2RGB)
img = cv2.cvtColor(cv2.imread(str(combine_path_tail(datadir, est["img_path"], 2))), cv2.COLOR_BGR2RGB)
if img.shape[0] == 1080 and img.shape[1] == 1920:
    Image.fromarray(overlay_img_alpha(img, maskalpha, alpha=0.3)).crop(pilcrop).save(tmpdir / f"rgbres_{datasetname}.png")
else:
    Image.fromarray(overlay_img_alpha(img, maskalpha, alpha=0.3)).save(tmpdir / f"rgbres_{datasetname}.png")
alpha = 0.6
img = cv2.addWeighted(img, alpha, rgb_primoverlay, 1 - alpha, 0)
# wireframewhere = np.where(rgb_wireframe[..., 3] > 0)
# img[wireframewhere] = rgb_wireframe[wireframewhere][..., :3]
img = overlay_img_alpha(img, rgb_wireframe)
mesh_wireframe, _, _ = render_models(cam, [recolor_mesh(mesh, black)], [obj2cam], wireframe=True, flags=pyrender.RenderFlags.RGBA | pyrender.RenderFlags.FLAT)
mesh_wireframe[mesh_wireframe[..., 3] > 0, 3] = 255
img = overlay_img_alpha(img, mesh_wireframe, alpha=0.3)
if img.shape[0] == 1080 and img.shape[1] == 1920:
    Image.fromarray(img).crop(pilcrop).save(tmpdir / f"rendered_overlay_{datasetname}.png")
else:
    Image.fromarray(img).save(tmpdir / f"rendered_overlay_{datasetname}.png")

# gt code
maskrgb, _, _ = render_models(cam, [mesh, plane], [obj2camgt, floor2camgt], flags=pyrender.RenderFlags.FLAT)
objmask = np.all(maskrgb == [255, 0, 0], axis=-1)
rgb_wireframe, _, _ = render_models(cam, [recolor_mesh(plane, black)], [floor2camgt], wireframe=True, flags=pyrender.RenderFlags.RGBA | pyrender.RenderFlags.FLAT)
rgb_wireframe = rgb_wireframe * ~objmask[..., None]
rgb_wireframe[rgb_wireframe[..., 3] > 0, 3] = 255
rgb_primoverlay, _, _ = render_models(cam, [mesh, plane], [obj2camgt, floor2camgt], light_intensity=50, wireframe=False)
# Image.fromarray(rgb_wireframe).save(tmpdir / "rendered_wireframe_gt.png")
# Image.fromarray(rgb_primoverlay).save(tmpdir / "rendered_gt.png")
# img = cv2.cvtColor(cv2.imread(est["img_path"]), cv2.COLOR_BGR2RGB)
img = cv2.cvtColor(cv2.imread(str(combine_path_tail(datadir, est["img_path"], 2))), cv2.COLOR_BGR2RGB)
alpha = 0.6
img = cv2.addWeighted(img, alpha, rgb_primoverlay, 1 - alpha, 0)
# wireframewhere = np.where(rgb_wireframe[..., 3] > 0)
# img[wireframewhere] = rgb_wireframe[wireframewhere][..., :3]
img = overlay_img_alpha(img, rgb_wireframe)
mesh_wireframe, _, _ = render_models(cam, [recolor_mesh(mesh, black)], [obj2camgt], wireframe=True, flags=pyrender.RenderFlags.RGBA | pyrender.RenderFlags.FLAT)
mesh_wireframe[mesh_wireframe[..., 3] > 0, 3] = 255
img = overlay_img_alpha(img, mesh_wireframe, alpha=0.3)
if img.shape[0] == 1080 and img.shape[1] == 1920:
    Image.fromarray(img).crop(pilcrop).save(tmpdir / f"rendered_overlay_{datasetname}_gt.png")
else:
    Image.fromarray(img).save(tmpdir / f"rendered_overlay_{datasetname}_gt.png")

## visualizing mask

In [ ]:
idx = 4
singlemasksinfo = masksinfo[idx]
singlemasksinfo

In [ ]:
bestidx = singlemasksinfo["best_idx"]
# [x_min, y_min, x_max, y_max]
bbox = list(map(int, singlemasksinfo["bboxes"][bestidx]))
# mask = np.array(Image.open(singlemasksinfo["mask_paths"][bestidx]).convert("L"))
mask = np.array(Image.open(combine_path_tail(resrootdir, singlemasksinfo["mask_paths"][bestidx], 4)).convert("L"))
maskbool = mask > 0
# img = np.array(Image.open(singlemasksinfo["img_path"]).convert("RGBA"))
img = np.array(Image.open(combine_path_tail(datadir, singlemasksinfo["img_path"], 2)).convert("RGBA"))

In [ ]:
maskalpha = np.zeros((*img.shape[:2], 4), dtype=np.uint8)
maskalpha[maskbool, 0:3] = [255, 0, 0]
maskalpha[maskbool, 3] = 255
maskalpha[~maskbool] = img[~maskbool]
alpha = 0.5
overlayed = cv2.addWeighted(img, alpha, maskalpha, 1 - alpha, 0)
overlayed = cv2.rectangle(overlayed, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (255, 197, 71, 255), 10)
tmp = Image.fromarray(overlayed)
tmp.save(tmpdir / f"mask_overlay_{datasetname}_{idx}.png")
tmp

## 3d reconstruction + multiview visualization thing + scale ambiguity visualization

In [ ]:
# Tworld_obj = T_from_blender(gtinfo[datasetname]["R"], gtinfo[datasetname]["t"], scale=gtinfo[datasetname]["scalefactor"])
scale = 1 / gtinfo[datasetname]["scalefactor"]
T_zup = T_from_blender(gtinfo[datasetname]["R_floor"], gtinfo[datasetname]["t_floor"], scale=1).inv

masks = []
for i, maskinfo in enumerate(masksinfo):
    maskpath = Path(maskinfo["mask_paths"][maskinfo["best_idx"]])
    maskpath = combine_path_tail(resrootdir, maskpath, 4)
    masks.append(np.array(Image.open(maskpath).convert("L")))
masks = np.array(masks)

densepc_zup = scale_pc(T_zup @ densepc, scale)
cams_zup = scale_cams(cams.apply_transform(T_zup), scale)
segidxs = segment_pc_from_masks(densepc_zup, masks, cams_zup, min_ratio=1/2)

# densepc_zup_unscaled = T_zup @ densepc
# cams_zup_unscaled = cams.apply_transform(T_zup)
densepc_zup_unscaled = scale_pc(T_zup @ densepc, 0.4)
cams_zup_unscaled = scale_cams(cams.apply_transform(T_zup), 0.4)

multiview consistency plot

In [ ]:
# multiview consistency plot
cam1 = cams_zup[0]
cam2 = cams_zup[-1]
# foundposeest1 = list(filter(lambda x: Path(x["img_path"]).stem == imgpaths[0].stem, foundposeests))[1]
foundposeest1 = list(filter(lambda x: Path(x["img_path"]).stem == imgpaths[0].stem, foundposeests))[0]
foundposeT1 = v3d.Transform(R=foundposeest1["R"], t=foundposeest1["t"])
T_objworld1 = cam1.world_from_cam @ foundposeT1
# foundposeest2 = list(filter(lambda x: Path(x["img_path"]).stem == imgpaths[-1].stem, foundposeests))[3]
foundposeest2 = list(filter(lambda x: Path(x["img_path"]).stem == imgpaths[-1].stem, foundposeests))[0]
foundposeT2 = v3d.Transform(R=foundposeest2["R"], t=foundposeest2["t"])
T_objworld2 = cam2.world_from_cam @ foundposeT2
# v3d.make_fig([densepc_zup, cam1, cam2, get_line3d_trace(cams_zup.world_from_cam.t, markersize=3, linewidth=3), T_objworld @ meshpc])
_, wiretrace1 = get_trimesh_traces(mesh.copy().apply_transform(T_objworld1.matrix4x4), wirecolor="rgba(255, 0, 0, 150)")
_, wiretrace2 = get_trimesh_traces(mesh.copy().apply_transform(T_objworld2.matrix4x4), wirecolor="rgba(0, 0, 255, 150)")
xx, yy, zz = get_plane_zup(densepc_zup.p, n=6, z=0, square_grid=True)
planetrace = go.Surface(x=xx, y=yy, z=zz, opacity=0.3, colorscale="purples")
planeoutlinetrace = get_surface_line_traces(xx, yy, zz)
fig = v3d.make_fig([
    densepc_zup,
    cam1,
    cam2.make_traces()[0].update(marker={"color": "rgb(0, 0, 255)"}),
    get_line3d_trace(cams_zup.world_from_cam.t, markersize=3, linewidth=3, linecolor="#ab63fa"),
    wiretrace1,
    wiretrace2,
    # planetrace,
    # *planeoutlinetrace,
])
fig.update_layout(
    showlegend=False,
    margin={'t':0,'l':0,'b':0,'r':0},
    plot_bgcolor="rgba(0, 0, 0, 0)",
    paper_bgcolor="rgba(0, 0, 0, 0)",
)
config = {
    "toImageButtonOptions": {
        "format": "png", # one of png, svg, jpeg, webp
        "width": 1920,
        "height": 1080,
        "filename": f"multiview3d-2cams-{datasetname}",
        # 'scale': 10,
    }
}
fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
fig.show(config=config)

# /scratch/jeyan/barreldata/tmp/newplot.png
# /Users/jerry/Projects/ms-stuff/barrel-playground/tmp/newplot.png

In [ ]:
cam_wireframes = [None] * len(cams_zup)
img = np.array(Image.open(imgpaths[0]).convert("RGBA"))
# objwireframe, _, _ = render_models(cam1, [recolor_mesh(mesh, [255, 0, 0, 255]), recolor_mesh(mesh, [0, 0, 255, 255])], [T_objworld1, T_objworld2], light_intensity=5000, wireframe=True, flags=pyrender.RenderFlags.RGBA)
objwireframe, _, _ = render_models(cam1, [recolor_mesh(mesh, [255, 0, 0, 255])], [T_objworld1], wireframe=True, flags=pyrender.RenderFlags.RGBA | pyrender.RenderFlags.FLAT)
wireframemask = objwireframe[..., 3] > 0
objwireframe[wireframemask, 3] = 255
# objwireframe[~wireframemask] = img[~wireframemask]
cam1_wireframe = Image.fromarray(overlay_img_alpha(img, objwireframe, alpha=0.5)).convert("RGB")
cam1_wireframe.save(tmpdir / f"cam1_wireframe_{datasetname}.png")
cam_wireframes[0] = cam1_wireframe
cam1_wireframe

In [ ]:
img = np.array(Image.open(imgpaths[-1]).convert("RGBA"))
# objwireframe, _, _ = render_models(cam2, [recolor_mesh(mesh, [255, 0, 0, 255]), recolor_mesh(mesh, [0, 0, 255, 255])], [T_objworld1, T_objworld2], light_intensity=5000, wireframe=True, flags=pyrender.RenderFlags.RGBA)
objwireframe, _, _ = render_models(cam2, [recolor_mesh(mesh, [0, 0, 255, 255])], [T_objworld2], light_intensity=5000, wireframe=True, flags=pyrender.RenderFlags.RGBA)
wireframemask = objwireframe[..., 3] > 0
objwireframe[wireframemask, 3] = 255
# objwireframe[~wireframemask] = img[~wireframemask]
cam2_wireframe = Image.fromarray(overlay_img_alpha(img, objwireframe, alpha=0.5)).convert("RGB")
cam2_wireframe.save(tmpdir / f"cam2_wireframe_{datasetname}.png")
cam_wireframes[-1] = cam2_wireframe
cam2_wireframe

generic 3d segmented point cloud plot

In [ ]:
# generic 3d segmented point cloud plot
mask1, bbox1 = get_bestmask(imgpaths[0].stem, masksinfo)
mask2, bbox2 = get_bestmask(imgpaths[-1].stem, masksinfo)

def create_maskvis(img, mask, bbox):
    maskalpha = np.zeros((*mask.shape[:2], 4), dtype=np.uint8)
    maskalpha[mask > 0, 0:3] = [255, 0, 0]
    maskalpha[mask > 0, 3] = 255
    overlayed = overlay_img_alpha(np.array(img), maskalpha, alpha=0.5)
    overlayed = cv2.rectangle(overlayed, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (255, 197, 71, 255), 10)
    return overlayed    

imgoverlayed1 = Image.fromarray(create_maskvis(imgs[0], mask1, bbox1)).convert("RGB")
imgoverlayed2 = Image.fromarray(create_maskvis(imgs[-1], mask2, bbox2)).convert("RGB")

objpc = densepc_zup[segidxs]
objpc = objpc.replace(rgb=objpc.rgb * 0.7 + np.array([[255, 0, 0]]) * 0.3)
seafloorpc = densepc_zup[invert_idxs(segidxs, len(densepc_zup))]
fig = v3d.make_fig([
    objpc.make_traces()[0].update(marker={"size": 2}),
    seafloorpc.make_traces()[0].update(marker={"size": 2}),
    cam1.make_traces()[0].update(marker={"color": "rgb(255, 0, 0)"}, line={"width": 7}),
    cam2.make_traces()[0].update(marker={"color": "rgb(255, 0, 0)"}, line={"width": 7}),
    get_cam_imgsurf_trace(cam1, imgoverlayed1, res_scale=0.25),
    get_cam_imgsurf_trace(cam2, imgoverlayed2, res_scale=0.25),
    get_line3d_trace(cams_zup.world_from_cam.t, markersize=8, linewidth=10, linecolor="#ab63fa"),
])
fig.update_layout(
    showlegend=False,
    margin={'t':0,'l':0,'b':0,'r':0},
    plot_bgcolor="rgba(0, 0, 0, 0)",
    paper_bgcolor="rgba(0, 0, 0, 0)",
)
config = {
    "toImageButtonOptions": {
        "format": "png", # one of png, svg, jpeg, webp
        "width": 1920 * 2,
        "height": 1080 * 2,
        "filename": f"segpc-{datasetname}",
        # 'scale': 10,
    }
}
fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
fig.show(config=config)

# /scratch/jeyan/barreldata/tmp/newplot.png
# /Users/jerry/Projects/ms-stuff/barrel-playground/tmp/newplot.png

In [ ]:
imgoverlayed1.save(tmpdir / f"cam1_maskoverlay_{datasetname}.png")
imgoverlayed2.save(tmpdir / f"cam2_maskoverlay_{datasetname}.png")

fit object with axis to point cloud figure

In [ ]:
# fit object with axis to point cloud figure
densepc_zup_filt = densepc_zup[(densepc_zup.p[:, 0] < 4) & (densepc_zup.p[:, 0] > -3) & (densepc_zup.p[:, 1] < 3) & (densepc_zup.p[:, 1] > -2.5)]
densepc_zup_filt = densepc_zup
xx, yy, zz = get_plane_zup(densepc_zup_filt.p, n=6, z=0, square_grid=True)
planetrace = go.Surface(x=xx, y=yy, z=zz, opacity=0.3, colorscale="purples")
planeoutlinetraces = get_surface_line_traces(xx, yy, zz)
cam2world0 = cams_zup[0].world_from_cam
obj2cam0 = v3d.Transform(R=ests[0]["R"], t=ests[0]["t"])
obj2world = cam2world0 @ obj2cam0
estmeshworld = mesh.copy().apply_transform(obj2world.matrix4x4)
estmeshtraces = get_trimesh_traces(estmeshworld, surfcolor="rgb(245, 156, 39)")
axtraces = get_axes_traces(obj2world @ v3d.Transform.from_angle(x=0, y=0, z=-np.pi / 2), scale=1.5, linewidth=10)
fig = v3d.make_fig([
    densepc_zup_filt.make_traces()[0].update(marker={"size": 1.5}),
    planetrace,
    *planeoutlinetraces,
    *estmeshtraces,
    *axtraces,
])
fig.update_layout(
    showlegend=False,
    margin={'t':0,'l':0,'b':0,'r':0},
    plot_bgcolor="rgba(0, 0, 0, 0)",
    paper_bgcolor="rgba(0, 0, 0, 0)",
)
config = {
    "toImageButtonOptions": {
        "format": "png", # one of png, svg, jpeg, webp
        "width": 1920,
        "height": 1080,
        "filename": f"fitpc-{datasetname}",
        # 'scale': 10,
    }
}
fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
fig.show(config=config)

# /scratch/jeyan/barreldata/tmp/newplot.png
# /Users/jerry/Projects/ms-stuff/barrel-playground/tmp/newplot.png

scale correction plots

In [ ]:
obj2cams_gt = []
for i, gtposeinfo in enumerate(sorted(gtposes, key=lambda x: x["img_path"])):
    T = v3d.Transform(R=gtposeinfo["R"], t=gtposeinfo["t"])
    obj2cams_gt.append(T)
obj2cams_gt = dca.stack(obj2cams_gt)
obj2cams_est = []
# for i, estposeinfo in enumerate(filter(lambda x: x["hypothesis_id"] == "0", sorted(ests, key=lambda x: x["img_path"]))):
for i, estposeinfo in enumerate(filter(lambda x: x["hypothesis_id"] == "0", sorted(foundposeests, key=lambda x: x["img_path"]))):
    T = v3d.Transform(R=estposeinfo["R"], t=estposeinfo["t"])
    obj2cams_est.append(T)
obj2cams_est = dca.stack(obj2cams_est)

In [ ]:
len(cams_zup_unscaled)

In [ ]:
mesh_camfront_unscaledpc = (cams_zup_unscaled.world_from_cam @ obj2cams_est @ meshpc[None, ...])
colors = (255 * cmapvals(np.arange(len(cams_zup_unscaled)), cmap="tab20")).astype(np.uint8)
colors[0] = [255, 0, 0]
colors[-1] = [0, 0, 255]
traces = []
skip = 13
camscale = 1.5
for i, cam_zup_unscaled in enumerate(cams_zup_unscaled):
    if i % skip != 0:
        continue
    camtrace = cam_zup_unscaled.replace_fig_config(scale=camscale).make_traces()[0].update(line={"color": rgb2hex(colors[i])})
    # meshtrace = mesh_camfront_unscaledpc[i].replace(rgb=colors[i])
    _, wiretrace = get_trimesh_traces(mesh.copy().apply_transform((cams_zup_unscaled.world_from_cam[i] @ obj2cams_est[i]).matrix4x4), wirecolor=rgb2hex(colors[i]))
    imgsurf_trace = get_cam_imgsurf_trace(cam_zup_unscaled.replace_fig_config(scale=camscale), cam_wireframes[i], res_scale=0.25, opacity=0.7)
    traces.append(camtrace)
    traces.append(wiretrace)
    traces.append(imgsurf_trace)
fig = v3d.make_fig([
    *traces,
    densepc_zup_unscaled[densepc_zup_unscaled.p[:, 2] > -1][::2].make_traces()[0].update(marker={"size": 2}),
    get_line3d_trace(cams_zup_unscaled.world_from_cam.t, markersize=3, linewidth=5, linecolor="#ab63fa"),
])
plotlycam = {
    'center': {'x': 0, 'y': 0, 'z': 0},
    'eye': {'x': 1.7135838058633417, 'y': -1.2023497067709676, 'z': 0.5527076287791509},
    'projection': {'type': 'perspective'},
    'up': {'x': 0, 'y': 0, 'z': 1},
}
fig.update_layout(
    scene_camera=plotlycam,
    showlegend=False,
    # margin={'t':0,'l':0,'b':0,'r':0},
    scene={
        # "xaxis": {"titlefont": {"size": 40}},
        # "yaxis": {"titlefont": {"size": 40}},
        "zaxis": {"nticks": 5},
    },
    font={"size": 15},
)
config = {
    "toImageButtonOptions": {
        "format": "png", # one of png, svg, jpeg, webp
        # "width": 1920,
        # "height": 1080,
        "filename": f"colmap-unscaled-{datasetname}",
        'scale': 2,
    }
}
# fig.write_image(tmpdir / f"colmap-unscaled-{datasetname}.png", scale=2)
fig.show(config=config)

In [ ]:
mesh_camfront_pc = (cams_zup.world_from_cam @ obj2cams_est @ meshpc[None, ...])
traces = []
for i, cam_zup in enumerate(cams_zup):
    if i % skip != 0:
        continue
    camtrace = cam_zup.make_traces()[0].update(line={"color": rgb2hex(colors[i])})
    # meshtrace = mesh_camfront_pc[i].replace(rgb=colors[i])
    _, wiretrace = get_trimesh_traces(mesh.copy().apply_transform((cams_zup.world_from_cam[i] @ obj2cams_est[i]).matrix4x4), wirecolor=rgb2hex(colors[i]))
    imgsurf_trace = get_cam_imgsurf_trace(cam_zup, cam_wireframes[i], res_scale=0.25, opacity=0.5)
    traces.append(camtrace)
    traces.append(wiretrace)
    traces.append(imgsurf_trace)
fig = v3d.make_fig([
    *traces,
    densepc_zup[densepc_zup.p[:, 2] > -1][::2].make_traces()[0].update(marker={"size": 2}),
    get_line3d_trace(cams_zup.world_from_cam.t, markersize=3, linewidth=3, linecolor="#ab63fa")
])
plotlycam = {
    'center': {'x': 0, 'y': 0, 'z': 0},
    'eye': {'x': 1.7135838058633417, 'y': -1.2023497067709676, 'z': 0.5527076287791509},
    'projection': {'type': 'perspective'},
    'up': {'x': 0, 'y': 0, 'z': 1},
}
fig.update_layout(
    scene_camera=plotlycam,
    showlegend=False,
    margin={'t':0,'l':0,'b':0,'r':0},
    scene={
        "xaxis": {"nticks": 7, "title": {"text": "x (m)"}},
        "yaxis": {"nticks": 7, "title": {"text": "y (m)"}},
        "zaxis": {"nticks": 5, "title": {"text": "z (m)"}},
    },
    font={"size": 15},
)
config = {
    "toImageButtonOptions": {
        "format": "png", # one of png, svg, jpeg, webp
        # "width": 1920 * 2,
        # "height": 1080 * 2,
        "filename": f"colmap-scaled-{datasetname}",
        'scale': 2,
    }
}
# fig.write_image(tmpdir / f"colmap-scaled-{datasetname}.png", scale=2)
fig.show(config=config)


## ICP + rotation visualization

In [ ]:
cameras = cams
use_coarse = True
scenepts = densepc.replace()
masks = []
for i, maskinfo in enumerate(masksinfo):
    maskpath = Path(maskinfo["mask_paths"][maskinfo["best_idx"]])
    maskpath = combine_path_tail(resrootdir, maskpath, 4)
    masks.append(np.array(Image.open(maskpath).convert("L")))
masks = np.array(masks)
objectmesh = mesh.copy()
objectsymmetries = get_symmetry_transformations(objinfo["barrelsingle-scaled.ply"], 0.01)


cams = cameras
# names, cameras are already assumed to be ordered and filtered
# depending on failed registration in COLMAP or SAM masks
name2cam = {name: cam for name, cam in zip(names, cameras)}
# foundpose results are used as reference for image ids
name2imgid = {}
name2imgpath = {}
obj2cams: v3d.Transform = []
# camera for each hypothesis. If multiple hypotheses for each image, there will be
# duplicate cameras in here
camhyps: v3d.Camera = []
# colmap usually can't reconstruct every camera pose, so we can only fit the
# foundpose results with a camera pose
for fres in foundposeests:
    imgpath = Path(fres["img_path"])
    name = imgpath.stem
    if name not in name2imgid.keys():
        name2imgid[name] = fres["img_id"]
    if name not in name2imgpath.keys():
        name2imgpath[name] = imgpath
    # could just use the "best" hypothesis. pretty often though, this hypothesis sucks.
    # valid_hyp = name in name2cam.keys() and fres["hypothesis_id"] == "0"
    valid_hyp = name in names
    if valid_hyp:
        camhyps.append(name2cam[name])
        if use_coarse:
            R = fres["R_coarse"]
            t = fres["t_coarse"]
        else:
            R = fres["R"]
            t = fres["t"]
        T = np.eye(4)
        T[:3, :3] = R
        T[:3, 3] = np.reshape(t, -1)
        obj2cams.append(v3d.Transform.from_matrix(T))
obj2cams = dca.stack(obj2cams)
camhyps = dca.stack(camhyps)

model, inlieridxs = ransac(
    camhyps.world_from_cam.matrix4x4,
    obj2cams.matrix4x4,
    fit_func=fitcams,
    loss_func=camloss,
    cost_func=camcost,
    samp_min=5,
    inlier_min=5,
    inlier_thres=0.15,
    max_iter=50,
    seed=0,
    relax_on_fail=True,
)

scalefactor = model.scale
camscaled = scale_cams(cams, scalefactor)
camhypsscaled = scale_cams(camhyps, scalefactor)
obj2worlds = camhypsscaled.world_from_cam @ obj2cams
obj2worldsinlier: v3d.Transform = obj2worlds[inlieridxs]

sceneptsscaled = scenepts.replace(p=scenepts.p * scalefactor)
segidxs = segment_pc_from_masks(sceneptsscaled, masks, camscaled, min_ratio=1/2)

# plane fit to seafloor
floormask = np.ones(len(sceneptsscaled), dtype=bool)
floormask[segidxs] = False
plane1 = pyrsc.Plane()
best_eq, best_inliers = plane1.fit(sceneptsscaled[floormask].p, thresh=0.05)
a, b, c, d = best_eq
normal = np.array([a, b, c])
R = get_axes_rot([0, 0, 1], normal)
planecent = [0, 0, -d / c]
Tmat = np.eye(4)
Tmat[:3, :3] = R
Tmat[:3, 3] = planecent
planeT = v3d.Transform.from_matrix(Tmat)
camzup = camscaled.apply_transform(planeT.inv)
if np.mean(camzup.world_from_cam.t[:, 2]) < 0:
    planeT = planeT @ v3d.Transform.from_matrix(np.array([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]))
    camzup = camscaled.apply_transform(planeT.inv)
    normal = -normal

# 3x points from sfm point cloud
meshsamp, _ = trimesh.sample.sample_surface(objectmesh, count=len(segidxs))
objinscenepts = sceneptsscaled[segidxs]
# simple outlier removal to see if this works
o3dobj = o3d.geometry.PointCloud()
o3dobj.points = o3d.utility.Vector3dVector(objinscenepts.p)
_, inlieridx = o3dobj.remove_statistical_outlier(nb_neighbors=50, std_ratio=2.0)
tmpT = []
icpTs = []
for i, obj2world in enumerate(obj2worldsinlier):
    # randomization test
    Q, _ = np.linalg.qr(np.random.uniform(0, 1, (3, 3)))
    perturbT = v3d.Transform(R=Q, t=np.random.uniform(-0.1, 0.1, (3)))
    # end
    samp_trf = obj2world @ meshsamp
    # source is sfm point cloud, since it is incomplete
    icpT = v3d.Transform.from_matrix(icp(objinscenepts.p[inlieridx], samp_trf, outlier_std=2.0))
    icpTs.append(icpT)
    tmpT.append(icpT.inv @ obj2world)
    # debug figure (this takes a while to generate)
    # meshpts = planeT.inv @ v3d.Point3d(p=samp_trf, rgb=[0, 0, 255])
    # meshicppts = planeT.inv @ v3d.Point3d(p=icpT.inv @ samp_trf, rgb=[0, 255, 0])
    # sceneobjzuppts = planeT.inv @ objinscenepts[inlieridx]
    # sceneobjzupoutlierpts = planeT.inv @ objinscenepts[invert_idxs(inlieridx, len(objinscenepts))]
    # sceneobjzupoutlierpts = sceneobjzupoutlierpts.replace(rgb=[255, 0, 0])
    # xycent = np.mean(sceneobjzuppts.p, axis=0)[:2]
    # centT = T_from_translation(-xycent[0], -xycent[1], 0)
    # icpfig = v3d.make_fig(
    #     [centT @ meshpts, centT @ meshicppts, centT @ sceneobjzuppts, centT @ sceneobjzupoutlierpts],
    # num_samples_point3d=1000)
    # icpfig.write_image(icpdebugdir / f"icpout_{str(i).zfill(4)}.png")
icpTs = dca.stack(icpTs)
obj2worldsinlier = dca.stack(tmpT)

quatsinlier = quaternion.from_rotation_matrix(obj2worldsinlier.R)
ref = quatsinlier[0]
quatssymd = [ref]
for otherquat in quatsinlier[1:]:
    best = closest_quat_sym(ref, otherquat, objectsymmetries)
    quatssymd.append(best)
quatssymd = np.array(quatssymd)
obj2worldsinliersym = obj2worldsinlier.replace(R=quaternion.as_rotation_matrix(quatssymd))

qmeanransac, qinliers = ransac(quatssymd, fit_func=qmean, loss_func=qloss, cost_func=qcost, samp_min=5, inlier_min=5, inlier_thres=0.2, max_iter=50, relax_on_fail=True)

meanT = v3d.Transform(R=quaternion.as_rotation_matrix(qmeanransac), t=np.mean(obj2worldsinliersym.t, axis=0))
quatfig = v3d.make_fig(*get_axes_traces(obj2worldsinliersym, scale=0.5), *get_axes_traces(meanT, linewidth=10))

# burial ratio by fitting plane to floor point cloud
T_zup = planeT.inv @ meanT
meshzup = objectmesh.copy().apply_transform(T_zup.matrix4x4)
mesh_zvals = meshzup.vertices[:, 2]
zmin, zmax = np.min(mesh_zvals), np.max(mesh_zvals)
if zmin >= 0:
    burial_ratio_z = 0
else:
    burial_ratio_z = abs(zmin) / (abs(zmin) + zmax)
slicedmesh = trimesh.intersections.slice_mesh_plane(meshzup, [0, 0, 1], [0, 0, 0], cap=True)
burial_ratio_vol = 1 - slicedmesh.volume / objectmesh.volume
burial_depth = abs(zmin)

# visualization of fitted scene
meshzuppts = v3d.Point3d(p=meshzup.vertices)
scenezuppts = planeT.inv @ sceneptsscaled
aggfig = v3d.make_fig([scenezuppts, meshzuppts, camzup])

plane2camfit = camscaled.world_from_cam.inv @ planeT[..., None]
obj2camfit = camscaled.world_from_cam.inv @ meanT[..., None]
estposes = []
for name, obj2cam, plane2cam in zip(names, obj2camfit, plane2camfit):
    posedata = {
        "img_path": str(name2imgpath[name]),
        "img_id": name2imgid[name],
        "hypothesis_id": "0",
        "R": obj2cam.R.tolist(),
        "t": obj2cam.t.tolist(),
        "R_floor": plane2cam.R.tolist(),
        "t_floor": plane2cam.t.tolist(),
    }
    estposes.append(posedata)

In [ ]:
invert_idxs(segidxs, len(sceneptsscaled))
objpts = (planeT.inv @ sceneptsscaled)[segidxs]
objpts = objpts.replace(rgb=objpts.rgb * 0.6 + np.array([[255, 0, 0]]) * 0.4)
fig = v3d.make_fig([objpts, (planeT.inv @ sceneptsscaled)[invert_idxs(segidxs, len(sceneptsscaled))]])
fig.update_layout(
    showlegend=False,
)
config = {
    "toImageButtonOptions": {
        "format": "png", # one of png, svg, jpeg, webp
        "filename": "pcvis",
        "width": 1920,
        "height": 1080,
        "scale": 1,
    }
}
fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
fig.show(config=config)

In [ ]:
idx = 25
icpT = icpTs[idx]
obj2world = obj2worldsinlier[idx]

# sceneobjzuppts = planeT.inv @ objinscenepts[inlieridx]
sceneobjzuppts = planeT.inv @ objinscenepts
# sceneobjzupoutlierpts = planeT.inv @ objinscenepts[invert_idxs(inlieridx, len(objinscenepts))]
# sceneobjzupoutlierpts = centT @ sceneobjzupoutlierpts.replace(rgb=[181, 103, 0])
# sceneobjzuppts = centT @ sceneobjzuppts.replace(rgb=[255, 0, 0])
xycent = np.mean(sceneobjzuppts.p, axis=0)[:2]
centT = T_from_translation(-xycent[0], -xycent[1], 0)

# samp_trf = obj2world @ meshpc
# meshpts = centT @ planeT.inv @ samp_trf.replace(rgb=[0, 0, 255])
# undo ICP transform
_, wireframetrace = get_trimesh_traces(
    mesh.copy().apply_transform((centT @ planeT.inv @ icpT @ obj2world).matrix4x4),
    wirecolor="rgb(255, 0, 0)",
    wirewidth=5,
)
# meshicppts = centT @ planeT.inv @ (icpT.inv @ samp_trf).replace(rgb=[0, 255, 0])
_, wireframeicptrace = get_trimesh_traces(
    mesh.copy().apply_transform((centT @ planeT.inv @ obj2world).matrix4x4),
    wirecolor="rgb(0, 0, 255)",
    wirewidth=5,
)
floorpts = centT @ planeT.inv @ sceneptsscaled[invert_idxs(segidxs, len(sceneptsscaled))]
floorpts = floorpts.replace(rgb=[255, 200, 200])
icpfig = v3d.make_fig([
    wireframetrace,
    wireframeicptrace,
    get_resized_pc_trace(centT @ sceneobjzuppts, 5),
    # sceneobjzupoutlierpts
], num_samples_point3d=100000)
icpfig.update_layout(
    showlegend=False,
)
config = {
    "toImageButtonOptions": {
        "format": "png", # one of png, svg, jpeg, webp
        "filename": f"icpvis-{idx}",
        "width": 1920,
        "height": 1920,
        "scale": 1,
    }
}
icpfig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
icpfig.show(config=config)

In [ ]:
objpts = (centT @ planeT.inv @ sceneptsscaled)[segidxs]
objpts = objpts.replace(rgb=objpts.rgb * 0.6 + np.array([[255, 0, 0]]) * 0.4)
objpts = objpts[::50]
floorpts = (centT @ planeT.inv @ sceneptsscaled)[invert_idxs(segidxs, len(sceneptsscaled))]
xx, yy, zz = get_plane_zup(floorpts.p, n=7, square_grid=True)
planetrace = go.Surface(x=xx, y=yy, z=zz, opacity=0.3, colorscale="purples", showscale=False)
planeoutlinetraces = get_surface_line_traces(xx, yy, zz)
floorpts = floorpts[::10]
objtrace = objpts.make_traces()[0].update(
    marker={
        "size": 5
    }
)
floortrace = floorpts[floorpts.p[:, 2] > -1].make_traces()[0].update(
    marker={
        "size": 5
    }
)
facetrace, wireframetrace = get_trimesh_traces(
    mesh.copy().apply_transform((centT @ planeT.inv @ meanT).matrix4x4),
    surfcolor="rgba(64, 203, 222, 0.4)",
    wirecolor="rgba(0, 0, 0, 0.2)"
)
axrot = v3d.Transform.from_angle(x=np.pi, y=0, z=0)
newquatfig = v3d.make_fig(
    # *get_axes_traces(centT @ planeT.inv @ obj2worldsinliersym, scale=0.5),
    *get_axes_traces(centT @ planeT.inv @ meanT @ axrot, linewidth=40),
    objtrace,
    floortrace,
    facetrace,
    wireframetrace,
    planetrace,
    *planeoutlinetraces
)
# newquatfig = v3d.make_fig(*get_axes_traces(centT @ planeT.inv @ obj2worldsinliersym, scale=0.5), trace)
newquatfig.update_layout(
    showlegend=False,
)
config = {
    "toImageButtonOptions": {
        "format": "png", # one of png, svg, jpeg, webp
        "filename": "quatavgvis",
        "width": 1920 * 2,
        "height": 1080 * 2,
        "scale": 1,
    }
}
newquatfig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
newquatfig.show(config=config)

In [ ]:
sceneobjzuppts = planeT.inv @ objinscenepts[inlieridx]
sceneobjzuppts = sceneobjzuppts.replace(rgb=[255, 0, 0])
xycent = np.mean(sceneobjzuppts.p, axis=0)[:2]
centT = T_from_translation(-xycent[0], -xycent[1], 0)
crap = centT @ planeT.inv @ sceneptsscaled[invert_idxs(segidxs, len(sceneptsscaled))]
crap = crap.replace(rgb=[255, 0, 0])
shit = centT @ planeT.inv @ sceneptsscaled[segidxs]
shit = shit.replace(rgb=[255, 200, 200])
xx, yy, zz = get_plane_zup(crap.p, n=10, z=0, square_grid=True)
planetrace = go.Surface(x=xx, y=yy, z=zz, opacity=0.5, colorscale="purples")
planeoutlinetrace = get_surface_line_traces(xx, yy, zz)
zupray = get_ray_trace([0, 1, 0], [0, 0, 1], length=1.5, width=5, color="#10c900")
trfmesh = mesh.copy().apply_transform((centT @ planeT.inv @ obj2world).matrix4x4)
facecols = ["#c98000"] * len(trfmesh.faces)
meshfig = ff.create_trisurf(x=trfmesh.vertices[:, 0], y=trfmesh.vertices[:, 1], z=trfmesh.vertices[:, 2], simplices=trfmesh.faces, color_func=facecols, show_colorbar=False)
planefitfig = v3d.make_fig(
    [crap, shit.make_traces()[0].update(marker={"color": "rgba(200, 1, 1, 0.005)"}), planetrace, *planeoutlinetrace, zupray, *meshfig.select_traces()],
num_samples_point3d=1000)
planefitfig.update_layout(
    showlegend=False,
)
# 1200/450
config = {
    "toImageButtonOptions": {
        "format": "png", # one of png, svg, jpeg, webp
        "filename": "planefitvis",
        "width": 2400,
        "height": 900,
        "scale": 1,
    }
}
planefitfig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
planefitfig.show(config=config)

## map of burial fractions

In [ ]:
multiview_fitted = True
pose_type = "coarse"
use_icp = True
# reconstr_type = "colmap"
reconstr_type = None
metricsfiltdf = filtermetrics(metricsdf, multiview_fitted=multiview_fitted, pose_type=pose_type, use_icp=use_icp, reconstr_type=reconstr_type)
burialdepths = metricsfiltdf["burial_depth"].values
burialdepths_gt = metricsfiltdf["burial_depth_gt"].values
burialdepths_err = metricsfiltdf["burial_error_depth"].values
lats = metricsfiltdf["lat"].values
lons = metricsfiltdf["lon"].values

def within_domain(lats, lons, domain):
    latmin, latmax = domain["S"], domain["N"]
    lonmin, lonmax = domain["W"], domain["E"]
    return (latmin <= lats) & (lats <= latmax) & (lonmin <= lons) & (lons <= lonmax)

### pcolor plots

In [ ]:
from sklearn.cluster import KMeans, SpectralClustering, MeanShift, AgglomerativeClustering

positions = np.array([lons, lats]).T
# clustermodel = KMeans(n_clusters=5, random_state=0).fit(positions)
# clustermodel = SpectralClustering(n_clusters=5, random_state=0).fit(positions)
# clustermodel = MeanShift(bandwidth=10).fit(positions)
clustermodel = AgglomerativeClustering(n_clusters=None, distance_threshold=0.007).fit(positions)
posclass = clustermodel.labels_
plt.scatter(lons, lats, c=posclass)

In [ ]:
domain = generate_domain(lats, lons, padding=0.01)
xi = np.linspace(domain["W"], domain["E"], 500)
yi = np.linspace(domain["S"], domain["N"], 500)
grid = np.meshgrid(xi, yi)
fig, ax = get_carree_axis(domain=domain, land=True)
ax.add_feature(cartopy.feature.OCEAN, facecolor="#dbf7ff")
get_carree_gl(ax)
for i in range(clustermodel.n_clusters_):
    mask = posclass == i
    if np.sum(mask) <= 2:
        continue
    zi = scipy.interpolate.griddata(np.array([lons, lats]).T[mask], burialdepths_gt[mask], grid, method="linear")
    pcmesh = ax.pcolormesh(xi, yi, zi)
# pcmesh = ax.pcolormesh(xi, yi, zi)
# sc = ax.scatter(lons, lats, color="red", edgecolor="k", s=4)
cb = plt.colorbar(pcmesh, label="Burial Depths (m)")
plt.show()

In [ ]:
# domain = generate_domain(lats, lons, padding=0.3)
domain = {
    "W": -118.6,
    "E": -118.2,
    "S": 33.33,
    "N": 33.62,
}
# insetdomain = generate_domain(lats, lons, padding=0.01)
insetdomain = {
    "W": -118.445,
    "E": -118.415,
    "S": 33.559,
    "N": 33.568,
}
vmin = min(np.min(burialdepths_gt), np.min(burialdepths))
vmax = max(np.max(burialdepths_gt), np.max(burialdepths))
xi = np.linspace(domain["W"], domain["E"], 1000)
yi = np.linspace(domain["S"], domain["N"], 1000)
grid = np.meshgrid(xi, yi)
zi = scipy.interpolate.griddata(np.array([lons, lats]).T, burialdepths_gt, grid, method="linear")
fig, ax = get_carree_axis(domain=domain, land=True)
ax.add_feature(cartopy.feature.OCEAN, facecolor="#dbf7ff")
get_carree_gl(ax)
for i in range(clustermodel.n_clusters_):
    mask = posclass == i
    if np.sum(mask) < 2:
        continue
    zi = scipy.interpolate.griddata(np.array([lons, lats]).T[mask], burialdepths_gt[mask], grid, method="linear")
    pcmesh = ax.pcolormesh(xi, yi, zi, vmin=vmin, vmax=vmax)
cb = plt.colorbar(pcmesh, label="Burial Depth (m)")
sc = ax.scatter(lons, lats, color="red", edgecolor="k", s=5, alpha=0.7)
axins = ax.inset_axes(
    [0.1, 0.05, 0.87, 0.68],
    xlim=(insetdomain["W"], insetdomain["E"]),
    ylim=(insetdomain["S"], insetdomain["N"]),
    xticklabels=[], yticklabels=[]
)
xi = np.linspace(insetdomain["W"], insetdomain["E"], 100)
yi = np.linspace(insetdomain["S"], insetdomain["N"], 100)
grid = np.meshgrid(xi, yi)
for i in range(clustermodel.n_clusters_):
    mask = posclass == i
    if np.sum(mask) <= 2:
        continue
    zi = scipy.interpolate.griddata(np.array([lons, lats]).T[mask], burialdepths_gt[mask], grid, method="linear")
    pcmesh = axins.pcolormesh(xi, yi, zi, vmin=vmin, vmax=vmax, edgecolors="face")
axins.scatter(lons, lats, color="red", edgecolor="k", s=3, alpha=0.7)
axins.set_facecolor("#dbf7ff")
ax.indicate_inset_zoom(axins, edgecolor="black")
fig.suptitle("Absolute burial depth GT")
# fig.savefig(tmpdir / "burialmapgt_pcolor.png", dpi=300, bbox_inches="tight")
# plt.show()

### scatter plots

In [ ]:
cmap = "plasma"
# domain = generate_domain(lats, lons, padding=0.3)
domain = {
    "W": -118.6,
    "E": -118.2,
    "S": 33.42,
    "N": 33.62,
}
# insetdomain = generate_domain(lats, lons, padding=0.01)
insetdomain = {
    "W": -118.445,
    "E": -118.415,
    "S": 33.559,
    "N": 33.568,
}
vmin = min(np.min(burialdepths_gt), np.min(burialdepths))
vmax = max(np.max(burialdepths_gt), np.max(burialdepths))
vmax = 1.0
fig, ax = get_carree_axis(domain=domain, land=True)
ax.add_feature(cartopy.feature.OCEAN, facecolor="#dbf7ff")
gl = get_carree_gl(ax, lat_interval=0.05, lon_interval=None)
sc = ax.scatter(lons, lats, c=burialdepths_gt, s=2, alpha=1.0, vmin=vmin, vmax=vmax, cmap=cmap)
axins = ax.inset_axes(
    [0.1, 0.0, 0.87, 0.68],
    projection=ccrs.PlateCarree(),
    xlim=(insetdomain["W"], insetdomain["E"]),
    ylim=(insetdomain["S"], insetdomain["N"]),
    xticklabels=[], yticklabels=[]
)
get_carree_gl(axins, labels=False, lat_interval=0.003, lon_interval=0.01)
scin = axins.scatter(lons, lats, c=burialdepths_gt, s=5, alpha=1.0, vmin=vmin, vmax=vmax, cmap=cmap)
cb = plt.colorbar(sc, ax=ax, label="Burial Depth (m)", fraction=0.025, pad=0.04)
axins.set_facecolor("#dbf7ff")
ax.indicate_inset_zoom(axins, edgecolor="black")
plt.draw()
for label in gl.xlabel_artists:
    label.set_fontsize(12)
for label in gl.ylabel_artists:
    label.set_fontsize(12)
# fig.suptitle("Absolute burial depth GT")
fig.savefig(tmpdir / "burialmapgt.png", dpi=300, bbox_inches="tight")
# plt.savefig(tmpdir / "burialmapgt.png", dpi=300, bbox_inches="tight")
# plt.show()

In [ ]:
regionheight = haversine(insetdomain["S"], insetdomain["N"], insetdomain["W"], insetdomain["W"])
regionwidth = haversine(insetdomain["S"], insetdomain["S"], insetdomain["W"], insetdomain["E"])
((regionheight / 1000) * (regionwidth / 1000))

In [ ]:
fig, ax = get_carree_axis(domain=insetdomain, land=True)
ax.add_feature(cartopy.feature.OCEAN, facecolor="#dbf7ff")
gl = get_carree_gl(ax, lat_interval=0.003, lon_interval=0.01)
sc = ax.scatter(lons, lats, c=burialdepths, s=5, alpha=1.0, vmin=vmin, vmax=vmax, cmap=cmap)
cb = plt.colorbar(sc, ax=ax, label="Burial Depth (m)", fraction=0.025, pad=0.04)
cb.set_label("Burial Depth (m)", fontsize=12)
cb.ax.tick_params(labelsize=12)
plt.draw()
for label in gl.xlabel_artists:
    label.set_fontsize(12)
for label in gl.ylabel_artists:
    label.set_fontsize(12)
# fig.suptitle("Absolute burial depth predicted")
fig.savefig(tmpdir / "burialmappred.png", dpi=300, bbox_inches="tight")

In [ ]:
insetmask = within_domain(lats, lons, insetdomain)
errmin = np.min(burialdepths_err[insetmask])
errmax = np.max(burialdepths_err[insetmask])
fig, ax = get_carree_axis(domain=insetdomain, land=True)
ax.add_feature(cartopy.feature.OCEAN, facecolor="#dbf7ff")
gl = get_carree_gl(ax, lat_interval=0.003, lon_interval=0.01)
sc = ax.scatter(lons, lats, c=burialdepths_err, s=5, alpha=0.8, vmin=errmin, vmax=errmax, cmap=cmap)
cb = plt.colorbar(sc, ax=ax, label="Absolute Burial Depth Error (m)", fraction=0.025, pad=0.04)
cb.set_label("Absolute Burial Depth Error (m)", fontsize=12)
cb.ax.tick_params(labelsize=12)
plt.draw()
for label in gl.xlabel_artists:
    label.set_fontsize(12)
for label in gl.ylabel_artists:
    label.set_fontsize(12)
# fig.suptitle("Absolute burial depth predicted")
fig.savefig(tmpdir / "burialmaperr.png", dpi=300, bbox_inches="tight")

In [ ]:
rightregion = {"N": 33.569, "S": 33.564, "W": -118.42, "E": -118.41}
middleregion = {"S": 33.562, "N": 33.567, "W": -118.435, "E": -118.425}
rightmask = within_domain(lats, lons, rightregion)
middlemask = within_domain(lats, lons, middleregion)
rightdepths_gt = burialdepths_gt[rightmask]
middledepths_gt = burialdepths_gt[middlemask]
rightdepths = burialdepths[rightmask]
middledepths = burialdepths[middlemask]
righterrs = burialdepths_err[rightmask]
middleerrs = burialdepths_err[middlemask]
print(f"GT: (central region: mean {np.mean(middledepths_gt)} stddev {np.std(middledepths_gt, ddof=1)}, east region: mean: {np.mean(rightdepths_gt)} stddev {np.std(rightdepths_gt, ddof=1)})")
print(f"Predicted: (central region: mean {np.mean(middledepths)} stddev {np.std(middledepths, ddof=1)}, east region: mean {np.mean(rightdepths)} stddev {np.std(rightdepths, ddof=1)})")
print(f"count: (central: {np.sum(middlemask)}, east: {np.sum(rightmask)})")

In [ ]:
lookhere = filtermetrics(metricsdf, name="dive3-depthcharge-05-11", multiview_fitted=True, use_icp=True, pose_type="coarse").iloc[0]
# lookhere = filtermetrics(metricsdf, name="barrel2", multiview_fitted=True, use_icp=True, pose_type="coarse").iloc[0]
# lookhere = filtermetrics(metricsdf, name="dive6-smokefloat-10-17-26", multiview_fitted=True, use_icp=True, pose_type="coarse").iloc[0]
trace = go.Scatter(
    x=lons,
    y=lats,
    mode="markers",
    marker=dict(
        size=5,
    ),
)
tracesingle = go.Scatter(
    x=[lookhere["lon"]],
    y=[lookhere["lat"]],
    mode="markers",
    marker=dict(
        size=10,
    ),
)
go.Figure(data=[trace, tracesingle]).show()

In [ ]:
# plt.hist(burialdepths_gt, range=[0, 0.9], bins=20)
sns.set_theme(font_scale=1.3, style="ticks")
ax = sns.histplot({"Manual label": burialdepths_gt, "Predicted": burialdepths}, binrange=(0, 0.9), bins=20, element="step")
ax.set(xlabel="Burial depth (m)")
# ax.get_figure().savefig(tmpdir / "burialmapboth_hist.png", dpi=300, bbox_inches="tight")
ax.get_figure().savefig(tmpdir / "burialmapboth_hist.pdf", dpi=300, bbox_inches="tight", format="pdf")

In [ ]:
sns.set_theme(font_scale=1.3, style="ticks")
ax = sns.histplot(burialdepths_err, binrange=(0, 0.55), bins=20)
ax.set(xlabel="Absolute burial depth error (m)")
# ax.get_figure().savefig(tmpdir / "burialmaperr_hist.png", dpi=300, bbox_inches="tight")
ax.get_figure().savefig(tmpdir / "burialmaperr_hist.pdf", dpi=300, bbox_inches="tight", format="pdf")

In [ ]:
sns.set_theme(font_scale=1.3, style="ticks")
ax = sns.histplot(
    {"Burial depths manual label (m)": burialdepths_gt, "Burial depths predicted (m)": burialdepths},
    x="Burial depths manual label (m)",
    y="Burial depths predicted (m)",
    cbar=True, cbar_kws={"label": "Count"},
    bins=13,
    cmap=None,
)
ax.plot([0, 0.82], [0, 0.82], color="red")
plt.grid()
# ax.get_figure().suptitle("Burial depth ground truth vs predicted")
# ax.get_figure().savefig(tmpdir / "burialhist2d.png", dpi=300, bbox_inches="tight")
ax.get_figure().savefig(tmpdir / "burialhist2d.pdf", dpi=300, bbox_inches="tight", format="pdf")

In [ ]:
scipy.stats.pearsonr(burialdepths_gt, burialdepths)